In [ ]:
import pandas as pd
import os
import xlrd

# avoids WARNING *** file size (499584) not 512 + multiple of sector size (512) error
wb = xlrd.open_workbook('./product.xls', logfile=open(os.devnull, 'w'))
s0 = pd.read_excel(wb, engine='xlrd', sheet_name=0)

In [ ]:
s0.columns

In [42]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import json
%matplotlib inline

In [4]:
# https://data.austintexas.gov/Public-Safety/EMS-Spring-Festival-Season-2018-Incident-Details/q99r-2je5
# https://data.austintexas.gov/resource/ddqt-6p93.json
# SXSW 2018 was 3/9 - 3/18
ems = pd.read_csv('./EMS_-_Spring_Festival_Season_2018_-_Incident_Details.csv')

In [6]:
ems.head()

,Incident Key,Event Date,Event Day of Week,Incident Time,Location Longitude,Location Latitude,Responding Units,Dispositions,Transport Priorities,Patient Transports,Patient Refusals,Other Patient Outcomes,Total Patients,Patients Transported to CCP,Patients Transported From CPP,Incident Location,Current
0,1,03/13/2018,Tuesday,03/13/2018 03:00:00 PM,-97.739335,30.267068,SRU3,No Patient,NaN,0,0,0,0,0,0,"(30.267068, -97.739335)",NaN
1,2,03/13/2018,Tuesday,03/13/2018 04:00:00 PM,-97.738247,30.266782,"CCP1, SRU1, SRU3",Refusal,NaN,0,0,0,0,1,0,"(30.266782, -97.738247)",NaN
2,3,03/13/2018,Tuesday,03/13/2018 04:00:00 PM,-97.739335,30.267068,"CCP2, MTRC5, SRU5",Refusal,NaN,0,2,0,2,1,0,"(30.267068, -97.739335)",NaN
3,4,03/13/2018,Tuesday,03/13/2018 04:00:00 PM,-97.738025,30.267401,"CHP02, MCOT01, SRU3",No Patient,NaN,0,0,0,0,0,0,"(30.267401, -97.738025)",NaN
4,7,03/13/2018,Tuesday,03/13/2018 05:00:00 PM,-97.739313,30.267062,SRU1,Refusal,NaN,0,0,0,0,0,0,"(30.267062, -97.739313)",NaN


In [7]:
ems['Event Date'] = pd.to_datetime(ems['Event Date'])

In [41]:
ems

,Incident Key,Event Date,Event Day of Week,Incident Time,Location Longitude,Location Latitude,Responding Units,Dispositions,Transport Priorities,Patient Transports,Patient Refusals,Other Patient Outcomes,Total Patients,Patients Transported to CCP,Patients Transported From CPP,Incident Location,Current,lat,lon
0,1,2018-03-13,Tuesday,03/13/2018 03:00:00 PM,-97.739335,30.267068,SRU3,No Patient,NaN,0,0,0,0,0,0,"(30.267068, -97.739335)",NaN,30.267068,-97.739335
1,2,2018-03-13,Tuesday,03/13/2018 04:00:00 PM,-97.738247,30.266782,"CCP1, SRU1, SRU3",Refusal,NaN,0,0,0,0,1,0,"(30.266782, -97.738247)",NaN,30.266782,-97.738247
2,3,2018-03-13,Tuesday,03/13/2018 04:00:00 PM,-97.739335,30.267068,"CCP2, MTRC5, SRU5",Refusal,NaN,0,2,0,2,1,0,"(30.267068, -97.739335)",NaN,30.267068,-97.739335
3,4,2018-03-13,Tuesday,03/13/2018 04:00:00 PM,-97.738025,30.267401,"CHP02, MCOT01, SRU3",No Patient,NaN,0,0,0,0,0,0,"(30.267401, -97.738025)",NaN,30.267401,-97.738025
4,7,2018-03-13,Tuesday,03/13/2018 05:00:00 PM,-97.739313,30.267062,SRU1,Refusal,NaN,0,0,0,0,0,0,"(30.267062, -97.739313)",NaN,30.267062,-97.739313
5,8,2018-03-13,Tuesday,03/13/2018 06:00:00 PM,-97.737484,30.265535,"CCP3, SRU1",Refusal,NaN,0,1,0,1,1,0,"(30.265535, -97.737484)",NaN,30.265535,-97.737484
6,9,2018-03-13,Tuesday,03/13/2018 06:00:00 PM,-97.743171,30.267119,CCP1,Refusal,NaN,0,1,0,1,0,0,"(30.267119, -97.743171)",NaN,30.267119,-97.743171
7,10,2018-03-13,Tuesday,03/13/2018 07:00:00 PM,-97.743171,30.267119,CCP2,Refusal,NaN,0,1,0,1,0,0,"(30.267119, -97.743171)",NaN,30.267119,-97.743171
8,11,2018-03-13,Tuesday,03/13/2018 09:00:00 PM,-97.738247,30.266782,SRU4,Refusal,NaN,0,1,0,1,0,0,"(30.266782, -97.738247)",NaN,30.266782,-97.738247
9,12,2018-03-13,Tuesday,03/14/2018 12:00:00 AM,-97.738289,30.260114,"CCP2, DM07s, DM08s, SRU4",Dell Seton Med Ctr,C1-CHAR C1,1,0,0,1,1,1,"(30.260114, -97.738289)",NaN,30.260114,-97.738289


In [9]:
week_lookup = {
    0 : 'Mon',
    1 : 'Tue',
    2 : 'Wed',
    3 : 'Thurs',
    4 : 'Fri',
    5 : 'Sat',
    6 : 'Sun'
}

In [11]:
# mon = 0, sun = 6
ems.groupby(ems['Event Date'].dt.dayofweek.apply(lambda x: week_lookup[x])).size()

Event Date
Fri      45
Sat      48
Thurs    19
Tue      14
Wed      15
dtype: int64

In [ ]:
plt.subplots()

In [35]:
ems['lat'] = ems['Incident Location'].apply(lambda x: float(re.findall(r'-?\d+.\d+', x)[0]))
ems['lon'] = ems['Incident Location'].apply(lambda x: float(re.findall(r'-?\d+.\d+', x)[1]))

In [37]:
ems.head(1)

,Incident Key,Event Date,Event Day of Week,Incident Time,Location Longitude,Location Latitude,Responding Units,Dispositions,Transport Priorities,Patient Transports,Patient Refusals,Other Patient Outcomes,Total Patients,Patients Transported to CCP,Patients Transported From CPP,Incident Location,Current,lat,lon
0,1,2018-03-13,Tuesday,03/13/2018 03:00:00 PM,-97.739335,30.267068,SRU3,No Patient,NaN,0,0,0,0,0,0,"(30.267068, -97.739335)",NaN,30.267068,-97.739335


In [38]:
with open("./constants.json") as f:
    data = f.read()
    api_key = json.loads(data)['key']

gmaps.configure(api_key=api_key)
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(ems[['lat', 'lon']], weights=ems['Total Patients']))

In [39]:
fig

Figure(layout=FigureLayout(height='420px'))